# Assignment 1 - Linear Regression

In this assigment you will be coding for a Linear Regression task hands-on. (10 Points)

The notebook uses some popular libraries. If your environment is missing any of these libraries, you can install them using the following `pip` commands:

```bash
!pip install matplotlib seaborn scikit-learn


In [ ]:
import math

from sklearn.datasets import fetch_california_housing
import pandas as pd
from pandas.plotting import scatter_matrix
from scipy import stats
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
#make sizes bigger for readability
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 17})
plt.rcParams["figure.figsize"] = (12,12)

## Load and Explore Data

In [ ]:
# Load the California Housing dataset
housing = fetch_california_housing()
# Convert the dataset into a DataFrame
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target  # Add the target (median house value)

Number of Instances:

    20640
Number of Attributes:

    8 numeric, predictive attributes and the target
Attribute Information:

        MedInc median income in block group

        HouseAge median house age in block group

        AveRooms average number of rooms per household

        AveBedrms average number of bedrooms per household

        Population block group population

        AveOccup average number of household members

        Latitude block group latitude

        Longitude block group longitude



In [ ]:
display(df)

In [ ]:
#Explore data for missingness
print(df.info())

In [ ]:
#Check statistics of the data
print(df.describe())

In [ ]:
# Display the first few rows
print(df.head())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select multiple features for the correlation check
X_all = df[['MedInc', 'AveRooms', 'AveOccup', 'HouseAge', 'Latitude', 'Longitude']]

# Calculate correlation matrix
corr_matrix = X_all.corr()

# Visualize the correlation matrix
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Features')
plt.show()

# Note that correlation between Latitude and Longitude is coming from geographical location of California

In [ ]:
#display scatter_matrix also
fig = plt.figure()
scatter_matrix(df,figsize =(25,25),alpha=0.9,diagonal="kde",marker="o")

### 1. Residual Sum of Squares (RSS)

$$
RSS = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

Where:

$$ y_i \text{ is the actual value} $$

$$ \hat{y}_i \text{ is the predicted value} $$

$$ n \text{ is the number of observations} $$

---

### 2. Residual Standard Error (RSE)

$$
RSE = \sqrt{\frac{RSS}{n - p - 1}}
$$

Where:

$$ RSS \text{ is the Residual Sum of Squares} $$

$$ n \text{ is the number of observations} $$

$$ p \text{ is the number of predictors (excluding the intercept)} $$

---

### 3. t-statistic

$$
t = \frac{\hat{\beta_j}}{SE(\hat{\beta_j})}
$$

Where:

$$ \hat{\beta_j} \text{ is the estimated coefficient for predictor } j $$

$$ SE(\hat{\beta_j}) \text{ is the standard error of the estimated coefficient for predictor } j $$

---

### 4. p-value

$$
p = 2 \cdot (1 - T(\lvert t \rvert, df))
$$

Where:

$$ t \text{ is the t-statistic} $$

$$ df \text{ is the degrees of freedom, calculated as } n - p - 1 $$

$$ T \text{ is the CDF of the t-distribution} $$



## Relevant Metrics

<span style="color:red">Task 1: Fill the missing parts (#TODO) of metric computations</span> (1 Point Each, 3 Points)


In [ ]:
def compute_rss(y, y_pred):
    """
    Compute Residual Sum of Squares (RSS)
    y: array of true target values
    y_pred: array of predicted target values
    """
    rss = 0
    for y_elt, y_pred_elt in zip(y, y_pred):
        rss += math.pow(y_elt - y_pred_elt, 2)
    
    return rss


def compute_rse(y, y_pred, n, p):
    """
    Compute Residual Standard Error (RSE)
    y: array of true target values
    y_pred: array of predicted target values
    n: number of observations
    p: number of predictors
    """
    rss = compute_rss(y, y_pred)
    
    return math.sqrt(rss/(n-p-1))

def compute_pvalue(X, y, y_pred):
    '''
    Compute p-values for the coefficients of a linear regression model.
    
    X: array of features
    y: array of true target values
    y_pred: array of predicted target values
    return: p-values for each feature
    '''
    n, p = X.shape  # Number of observations (n) and number of predictors (p)
    
    # Compute RSS and RSE
    rss = compute_rss(y, y_pred)
    rse = compute_rse(y, y_pred, n, p)
    
    # # Add intercept (constant term) to the design matrix X
    X = np.c_[np.ones(n), X]
    
    # Calculate (X^T X)^-1
    XTX_inv = np.linalg.inv(np.dot(X.T, X))
    
    # Compute standard error (SE) for each coefficient
    se = np.sqrt(np.diagonal(rse ** 2 * XTX_inv))
    
    # Fit the model to compute the coefficients (betas)
    beta_hat = np.linalg.lstsq(X, y, rcond=None)[0]
    
    # Compute t-statistics for each coefficient
    t_stats = beta_hat / se
    
    degrees_of_freedom = n - p - 1

    # Compute p-values
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=degrees_of_freedom))
    
    return p_values


## Linear Regression with single predictor 

In [ ]:
# Select features and target
X = df[['AveRooms']]
#z-normalize the data for each column
X = (X - X.mean()) / X.std()
y = df['MedHouseVal']

# Split the data into training and testing sets (80% training, 20% testing) with a fixing seed that ensures same split every time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

independent_scaler = StandardScaler()
X_train = independent_scaler.fit_transform(X_train)
X_test = independent_scaler.transform(X_test)

# Create a linear regression model
model_1 = LinearRegression()

# Train the model
model_1.fit(X_train, y_train)

# Get the coefficients
print(f"Intercept (β0): {model_1.intercept_}")
print(f"Coefficients (β1, β2): {model_1.coef_}")

#Compute RSS for training data
y_pred = model_1.predict(X_train)

# Compute RSS
rss = compute_rss(y_train, y_pred)

# Calculate R-squared
r_squared_all = model_1.score(X_train, y_train)

# Compute the p-value
p_value = compute_pvalue(X_train, y_train, y_pred)

# Display the coefficients and p-values in a DataFrame
coefficients = np.concatenate([[model_1.intercept_], model_1.coef_])
p_values = np.concatenate([ p_value])

display(pd.DataFrame(pd.DataFrame({'features': ['intercept'] + list(X.columns), 'coefficients': coefficients, 'p-values': p_values})))
print(f"RSS (test data): {rss}")
print(f"R-squared (test data): {r_squared_all}")



<span style="color:red">Task 2: Use 'MedInc', 'AveRooms', 'AveOccup', 'HouseAge', 'Latitude', 'Longitude' as predictors.</span> (2 Points)


In [ ]:
# X_all = #TODO
y = df['MedHouseVal']

# Split the data into training and testing sets (80% training, 20% testing) with a fixing seed that ensures same split every time
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y, test_size=0.2, random_state=42)
independent_scaler = StandardScaler()
X_train_all = independent_scaler.fit_transform(X_train_all)
X_test_all = independent_scaler.transform(X_test_all)

# Fit the linear regression model
model_2 = LinearRegression()
model_2.fit(X_train_all, y_train_all)

# Predictions on the test set
y_pred_all = model_2.predict(X_test_all)

#Code this part
rss = compute_rss(y_test_all, y_pred_all)

# Calculate R-squared
r_squared_all = model_2.score(X_test_all, y_test_all)

# Compute the p-value
p_value = compute_pvalue(X_test_all, y_test_all, y_pred_all)

# Display the coefficients and p-values in a DataFrame
coefficients = np.concatenate([[model_2.intercept_], model_2.coef_])
p_values = np.concatenate([ p_value])

# pd.DataFrame({'features': ['intercept'] + list(X_all.columns), 'coefficients': coefficients, 'p-values': p_values})
display(pd.DataFrame({'features': ['intercept'] + list(X_all.columns), 'coefficients': coefficients, 'p-values': p_values}))
print(f"RSS (test data): {rss}")
print(f"R-squared (test data): {r_squared_all}")


<span style="color:red">Task 3: Try model performance on different K values by using the code below, observe the effect of very large K values which one would you pick?</span> (3 Points)


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

X_all = df[['MedInc', 'AveRooms', 'AveOccup', 'HouseAge', 'Latitude', 'Longitude']]
X_all = (X_all - X_all.mean()) / X_all.std()
y = df['MedHouseVal']


# Split the data into training and testing sets (80% training, 20% testing) with a fixing seed that ensures same split every time
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y, test_size=0.2, random_state=42)
independent_scaler = StandardScaler()
X_train_all = independent_scaler.fit_transform(X_train_all)
X_test_all = independent_scaler.transform(X_test_all)

#Fit the KNN model (you can tune 'n_neighbors' for optimal performance)
knn_model = KNeighborsRegressor(n_neighbors=1)
knn_model.fit(X_train_all, y_train_all)

#Make predictions on the test set
y_pred_knn = knn_model.predict(X_test_all)

#Compute RSS and R-squared
rss_knn = compute_rss(y_test, y_pred_knn)
r2_knn = r2_score(y_test_all, y_pred_knn)
print(f"KNN Model RSS: {rss_knn}")
print(f"KNN Model R-squared: {r2_knn}")


<span style="color:red">Task 4: Comment on R-squared and RSS values</span> (1 Point)

TODO

## Visualize results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error


# Make predictions on the test set for the first model (only AveRooms)
y_pred = model_1.predict(X_test)

# Make predictions on the test set for the second model (multiple features)
y_pred_all = model_2.predict(X_test_all)

# Make predictions using the KNN model
y_pred_knn = knn_model.predict(X_test_all)  # Use scaled features for KNN


plt.figure(figsize=(10, 6))

# Model 1: True vs Fitted (only AveRooms)
sns.scatterplot(x=y_pred, y=y_test, color='blue', label='Model 1 (AveRooms)', alpha=0.1)

# Model 2: True vs Fitted (multiple features)
sns.scatterplot(x=y_pred_all, y=y_test_all, color='red', label='Model 2 (Multiple features)', alpha=0.1)

# KNN Model: True vs Fitted
sns.scatterplot(x=y_pred_knn, y=y_test_all, color='green', label='KNN Model', alpha=0.1)

# Add perfect prediction line
plt.plot([min(y_pred_all), max(y_pred_all)], [min(y_test_all), max(y_test_all)], color='black', linestyle='--', label='Perfect Prediction')

# Labels and title
plt.xlabel('Fitted Values (Predicted)')
plt.ylabel('True Values')
plt.title('True vs Fitted Values for Model 1 and Model 2')
plt.legend()
plt.show()


<span style="color:red">Task 5: Compute residuals</span> (1 Point)

In [ ]:
### 2. Residuals vs Fitted

# Compute residuals #TODO
residuals_model_1 = None #TODO
residuals_model_2 = None #TODO 
residuals_knn = None #TODO 

plt.figure(figsize=(10, 6))

# Residuals vs Fitted for Model 1 (only AveRooms)
sns.scatterplot(x=y_pred, y=residuals_model_1, color='blue', label='Model 1 (AveRooms)', alpha=0.1)

# Residuals vs Fitted for Model 2 (multiple features)
sns.scatterplot(x=y_pred_all, y=residuals_model_2, color='red', label='Model 2 (Multiple features)', alpha=0.1)

# Residuals vs Fitted for KNN Model
sns.scatterplot(x=y_pred_knn, y=residuals_knn, color='green', label='KNN Model', alpha=0.1)

# Add horizontal line at 0 (perfect prediction residual)
plt.axhline(0, color='black', linestyle='--')

# Labels and title
plt.xlabel('Fitted Values (Predicted)')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values for Model 1 and Model 2')
plt.legend()
plt.show()
